1- IMPORTAÇÃO DE BIBLIOTECAS QUE SERÃO UTILIZADAS NO CHATBOT

In [15]:
# UNISUAM - Bangu #
# A2 - Inteligência Artificial #
# Professor: Charles Bastos #
# Integrantes: Matheus Sebastião e Rodrigo Fernandes # 
# ChatBot - Retirada de Dúvidas Bancário #

import os
import io
import pandas as pd

import json
import pickle
import nltk
import random
import numpy as np

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from keras.models import load_model

from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
intents = json.loads(open('chatbot_intents.json').read())
print(intents)

{'intents': [{'tag': 'saudacao', 'patterns': ['ola', 'bom dia', 'boa tarde', 'boa noite', 'good morning', 'hi', 'hello', 'eae', 'oi', 'ei', 'tudo bem', 'saudações', 'e ai', 'fala ai', 'coe', 'buenas noches', 'buenos dias', 'hey', 'koe', 'alo'], 'responses': ['Olá, sou seu assistente virtual, em que posso te ajudar?', 'Oi, tudo bem? Posso te ajudar?', 'Ei! Como vai? Como posso te ajudar?', 'Tudo bem com você? Posso te ajudar com algo?', 'Olá, posso te ajudar?', 'Opa, como vai? Em que posso te ajudar?', 'Olá, que bom ter você aqui! Como posso te ajudar?', 'Oii! Em que posso ajudá-lo?', 'Hey! Tudo bem? Digite sua duvida pra mim', 'Opa! Como vai você? Qual sua duvida?', 'Olá amigo! Com o que deseja que eu te ajude hoje?'], 'context': ['']}, {'tag': 'quem_e_voce', 'patterns': ['Qual seu nome', 'quem voce', 'quem é', 'como se chama'], 'responses': ['Meu nome é MatGo, seu assistente virtual bancário para retirada de dúvidas', 'Me chamo MatGo, sou seu robô pessoal para tirar suas dúvidas', 'Pr

In [17]:
words = []
documents = []

classes = [i['tag'] for i in intents['intents']]
ignore_words = ["!", "@", "#", "$", "%", "*", "?"]

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))

In [18]:
def clear_writing(writing):
    sentence_words = nltk.word_tokenize(writing)
    return [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

def bag_of_words(writing, words):
    sentence_words = clear_writing(writing)

    bag = [0]*len(words)
    for setence in sentence_words:
        for i, word in enumerate(words):
            if word == setence:
                bag[i] = 1

    return(np.array(bag))


def class_prediction(writing, model):
    prevision = bag_of_words(writing, words)
    response_prediction = model.predict(np.array([prevision]))[0]
    results = [[index, response] for index, response in enumerate(response_prediction) if response > 0.25]    

    if "1" not in str(prevision) or len(results) == 0 :
        results = [[0, response_prediction[0]]]

    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]


def get_response(intents, intents_json):
    tag = intents[0]['intent']
    list_of_intents = intents_json['intents']
    for idx in list_of_intents:
        if idx['tag'] == tag:
            result = random.choice(idx['responses'])
            break

    return result

In [19]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [20]:
print('words = ', words)
print('classes = ', classes)

words =  ['adicional', 'ai', 'alo', 'atual', 'aumentar', 'aumento', 'bem', 'bloquear', 'bloqueio', 'boa', 'bom', 'buenas', 'buenos', 'cadastrar', 'cartão', 'chama', 'coe', 'como', 'consulta', 'criar', 'crédito', 'data', 'de', 'desbloquear', 'desbloqueio', 'dia', 'disponivel', 'e', 'eae', 'ei', 'fala', 'fatura', 'good', 'hello', 'hey', 'hi', 'koe', 'liberar', 'limite', 'morning', 'noches', 'noite', 'nome', 'obter', 'oi', 'ola', 'pagar', 'qual', 'quem', 'saber', 'saudações', 'se', 'seu', 'solicitar', 'tarde', 'ter', 'total', 'trancar', 'tudo', 'validade', 'vence', 'vencimento', 'voce', 'é']
classes =  ['aumento_limite', 'bloquear_cartao', 'consulta_limite', 'desbloquear_cartao', 'outro_assunto', 'quem_e_voce', 'saudacao', 'solicitar_cartao_adicional', 'vencimento_cartao', 'vencimento_fatura']


In [21]:
training = []
output_empty = [0] * len(classes)
for document in documents:
    bag = []

    pattern_words = document[0]

    pattern_words = [lemmatizer.lemmatize( word.lower()) for word in pattern_words]

    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
x = list(training[:, 0])
y = list(training[:, 1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [22]:
len(x[0])

64

In [23]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

m = model.fit(np.array(x), np.array(y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', m)

print("Modelo criado com sucesso")

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


12/12 [==============================] - 1s 4ms/step - loss: 2.2813 - accuracy: 0.1964 
Epoch 2/200
12/12 [==============================] - 0s 7ms/step - loss: 2.1318 - accuracy: 0.3571
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 2.1059 - accuracy: 0.3571
Epoch 4/200
12/12 [==============================] - 0s 4ms/step - loss: 1.9334 - accuracy: 0.3571
Epoch 5/200
12/12 [==============================] - 0s 4ms/step - loss: 1.8777 - accuracy: 0.3571
Epoch 6/200
12/12 [==============================] - 0s 6ms/step - loss: 1.7218 - accuracy: 0.3571
Epoch 7/200
12/12 [==============================] - 0s 8ms/step - loss: 1.7228 - accuracy: 0.4107
Epoch 8/200
12/12 [==============================] - 0s 6ms/step - loss: 1.5958 - accuracy: 0.4643
Epoch 9/200
12/12 [==============================] - 0s 3ms/step - loss: 1.4570 - accuracy: 0.4643
Epoch 10/200
12/12 [==============================] - 0s 8ms/step - loss: 1.2994 - accuracy: 0.5714
Epoch 11/200
12/12 [

In [24]:
model = load_model('chatbot.h5')

In [25]:
def chatbot_response(msg):
    ints = class_prediction(msg, model)
    res = get_response(ints, intents)
    return res

In [28]:
nome = input("Qual seu nome? ")
print("Bem vindo(a)", nome)
print("Caso queira encerrar a conversa é só digitar 'Sair'")
msg = input("Agora digite uma mensagem para começar a conversar com o ChatBot: ")
while True:
  if msg == 'Sair' or msg == 'sair':
    break
  resposta = chatbot_response(msg)
  print(f'MatGo: {resposta}')
  msg = input(f'{nome}: ')

Qual seu nome? jfas
Bem vindo(a) jfas
Caso queira encerrar a conversa é só digitar 'Sair'
Agora digite uma mensagem para começar a conversar com o ChatBot: bom dia
MatGo: Olá amigo! Com o que deseja que eu te ajude hoje?
jfas: desbloqueio de cartão
MatGo: Para desbloquear seu cartão acesse o aplicativo e tenha seu cartão em mãos. Caso prefira, ligue para nossa central
jfas: sair
